In [6]:
from retrieve import retrieve
vector_store_path = 'vector_store'
EB_PATH = r'D:\Huan\Project\KHDL\model\ebmodel\bge-m3-ft-triplet'
re = retrieve(vector_store_path = vector_store_path, EMBEDDING_MODEL_PATH = EB_PATH)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


Load embedding model...  

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



Done
Load vector store...  Done


Đánh giá truy xuất trên 10 dữ liệu đầu tiên


In [27]:
import json

with open("qs_context.jsonl", "r", encoding='utf-8') as file:
    data = [json.loads(line) for line in file]
    
retrieved_contexts = [[j.page_content for j in re.similarity_search(query=i['Question'], top_k=10)] for i in data]


In [30]:
print(len(retrieved_contexts[1]))

10


In [32]:
import numpy as np

def calculate_mrr(retrieved_contexts, relevant):
    """Tính MRR@10."""
    for rank, context in enumerate(retrieved_contexts[:10], 1):
        if context == relevant:  # context đúng là context duy nhất trong relevant
            return 1 / rank
    return 0  # Nếu không tìm thấy kết quả đúng trong 10 truy xuất

def calculate_recall(retrieved_contexts, relevant):
    """Tính Recall@10."""
    relevant_retrieved = [context for context in retrieved_contexts[:10] if context == relevant]
    return len(relevant_retrieved) / 1  # Vì chỉ có 1 context đúng trong relevant


# Khởi tạo các danh sách để lưu kết quả
mrr_scores = []
recall_scores = []

# Tính toán các chỉ số cho mỗi câu hỏi
for i in range(len(data)):
    retrieved = retrieved_contexts[i]
    relevant = data[i]['Context']
    
    # Tính MRR@10
    mrr_scores.append(calculate_mrr(retrieved, relevant))
    
    # Tính Recall@10
    recall_scores.append(calculate_recall(retrieved, relevant))

# Tính giá trị trung bình của các chỉ số
mean_mrr = np.mean(mrr_scores)
mean_recall = np.mean(recall_scores)

print(f"Mean MRR@10: {mean_mrr:.4f}")
print(f"Mean Recall@10: {mean_recall:.4f}")


Mean MRR@10: 0.7552
Mean Recall@10: 0.9168
